In [4]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [5]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
data = Path('../Resources/applicants_data.csv')
applicant_data_df = pd.read_csv(data)

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Define features set X by selecting all columns but DEFAULT
X = applicant_data_df.drop(columns=["EIN","NAME"]).copy()


In [7]:
X.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
X.dtypes
# Create a list of categorical variables 
categorical_variables = list(X.dtypes[X.dtypes == "object"].index)

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [22]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(X[categorical_variables])
encoded_data

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [23]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

C:\Users\Klin\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
numerical_variable_df = X.drop(columns = categorical_variables)
numerical_variable_df
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame

encoded_df = pd.concat(
    [ numerical_variable_df,
     encoded_df
    ],
    axis =1
)

encoded_df

# Review the Dataframe
# YOUR CODE HERE

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [30]:
# Define features set X by selecting all columns but Attrition_Yes and Attrition_No
X = encoded_df.drop(columns=["IS_SUCCESSFUL"])

# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [32]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.columns)

# Review the number of features
number_input_features 



116

In [56]:
# Define the number of neurons in the output layer
number_output_neurons = 1


In [57]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features+number_output_neurons)//2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [58]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [42]:
# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(number_output_neurons, activation="sigmoid"))

In [59]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [60]:
nn.compile(
    loss = "binary_crossentropy",
    optimizer= "adam",
    metrics = ["accuracy"]
)

In [85]:
# Fit the model using 100 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5331 - accuracy: 0.7404
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7419
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5320 - accuracy: 0.7416
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5321 - accuracy: 0.7419
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5319 - accuracy: 0.7421
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5315 - accuracy: 0.7416
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7417
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 0.7421
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5311 - accuracy: 0.7417
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5317 - accuracy: 0.7416

In [86]:

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5625 - accuracy: 0.7307 - 268ms/epoch - 1ms/step
Loss: 0.5625374913215637, Accuracy: 0.7307288646697998


In [ ]:

# Set the model's file path
file_path = Path('Resources/Alphabetsoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)

In [87]:
# Alternative Model 1
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

In [88]:
number_input_features 

116

In [89]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [90]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1 )//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

58

In [91]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1 + number_output_neurons_A1 )//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2_A1

29

In [92]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer3_A1 = (hidden_nodes_layer2_A1 + number_output_neurons_A1 )//2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer3_A1

15

In [104]:

# Create the Sequential model instance
nn_A1 = Sequential()

In [105]:
# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn_A1.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu")
         )

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn_A1.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer2_A1, activation="relu")
         )
# add 3rd layer
nn_A1.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer3_A1, activation="relu")
         )
# Output layer
nn_A1.add(tf.keras.layers.Dense(
        units=1, activation="sigmoid")
         )
# Check the structure of the model
nn_A1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 58)                6786      
                                                                 
 dense_20 (Dense)            (None, 29)                1711      
                                                                 
 dense_21 (Dense)            (None, 15)                450       
                                                                 
 dense_22 (Dense)            (None, 1)                 16        
                                                                 
Total params: 8,963
Trainable params: 8,963
Non-trainable params: 0
_________________________________________________________________


In [106]:
# Fit the model using 50 epochs and the training data
nn_A1.compile(
    loss = "binary_crossentropy",
    optimizer= "adam",
    metrics = ["accuracy"]
)

In [107]:
# Fit the model using 50 epochs and the training data
model_A1 =  nn_A1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.5759 - accuracy: 0.7184
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7300
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7308
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7327
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5451 - accuracy: 0.7346
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7327
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7345
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5420 - accuracy: 0.7356
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5420 - accuracy: 0.7346
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7354

In [108]:
# Alternative Model 2
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [109]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [111]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = round((number_input_features * .666666), 0)

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

77.0

In [116]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  (hidden_nodes_layer1_A2 + number_output_neurons_A2) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2

39.0

In [117]:
nn_A2 = Sequential()

In [118]:
# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn_A2.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="sigmoid")
         )

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn_A2.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer2_A2, activation="sigmoid")
         )

# Output layer
nn_A2.add(tf.keras.layers.Dense(
        units=1, activation="sigmoid")
         )
# Check the structure of the model
nn_A2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 77)                9009      
                                                                 
 dense_27 (Dense)            (None, 39)                3042      
                                                                 
 dense_28 (Dense)            (None, 1)                 40        
                                                                 
Total params: 12,091
Trainable params: 12,091
Non-trainable params: 0
_________________________________________________________________


In [119]:
nn_A2.compile( loss = "binary_crossentropy",
    optimizer= "adam",
    metrics = ["accuracy"]
)
    

In [121]:
# Fit the model using 50 epochs and the training data
model_A2 =  nn_A2.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.5915 - accuracy: 0.7088
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5707 - accuracy: 0.7248
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5656 - accuracy: 0.7252
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5605 - accuracy: 0.7270
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7289
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7303
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7311
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7319
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7315
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7315

In [122]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5693 - accuracy: 0.7311 - 234ms/epoch - 873us/step
Loss: 0.5693169832229614, Accuracy: 0.7310787439346313


In [123]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =nn_A1.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.6984 - accuracy: 0.5292 - 370ms/epoch - 1ms/step
Loss: 0.6983767747879028, Accuracy: 0.5292128324508667


In [124]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5497 - accuracy: 0.7329 - 340ms/epoch - 1ms/step
Loss: 0.5496607422828674, Accuracy: 0.7329446077346802


In [ ]:
# Set the model's file path
file_path_A1 = Path('Resources/Alphabetsoup_A1.h5")
file_path_A2 = Path('Resources/Alphabetsoup_A2.h5")
# Export your model to a HDF5 file
nn_A1.save(file_path_A1)
nn_A2.save(file_path_A2)